In [ ]:
import sys
import re
import json
from collections import Counter

## kommentit.py
- Oheisella scriptillä poimitaan conllu-tiedostosta Suomi24-kommentti, sen annotaatiot ja metadatasta comment_id

- Tuloksena seuraavissa vaiheissa hyödynnettävä json-tiedosto

In [ ]:
def read_text(inp):
    id = None
    annotaatiot = []
    for line in inp:
        line = line.strip()
        if line.startswith("ID:"):
            if id is not None:
                yield (id, kommentit, annotaatiot)
            id = line
            kommentit = []
            annotaatiot = []
        elif id is None:
            id = line
        else:
            if line.startswith("['"):
              annotaatiot.append(line)
              continue
            else:
                if line.startswith("TEKSTI:"):
                    line = line.split("TEKSTI:", 1)
                    kommentti = line[1].strip()
                    kommentit.append(kommentti)

output_data = []

for id, kommentit, annotaatiot in read_text(sys.stdin):
    for kommentti in kommentit:
        comment_id = re.search(r'\s*id\s*=\s*["\']?(\d+:\d+)', id).group(1)
        output_data.append({"comment_id": comment_id, "comment": kommentti, "annot": annotaatiot})

with open("kommentit.json", "w") as output_file:
    json.dump(output_data, output_file, indent=4)


## murrepiirteet.py
- kommenttitiedostosta voidaan hakea murrepiirteitä regexien avulla
- esimerkkinä ts-vaihtelu, mutta jatkoa varten tätä tulisi laajentaa huomioimaan annotaatiot, jotta osumatarkkuus paranisi. Tällaisenaan mukana paljon roskaa.

In [ ]:
ts_variantti = r'\w+[aeiouyäö]ts[aeiouyäö]\w*$'
tt_variantti = r'\w+[aeiouyäö]tt[aeiouyäö]\w*$'
ht_variantti = r'\w+[aeiouyäö]ht[aeiouyäö]\w*$'

def search_patterns_in_comments(data):
    pattern_matches = {}

    for item in data:
        comment_id = item["comment_id"]
        comment = item["comment"]

        matches_ts = re.findall(ts_variantti, comment)
        matches_ht = re.findall(ht_variantti, comment)
        matches_tt = re.findall(tt_variantti, comment)

        if matches_ts or matches_ht or matches_tt:
            pattern_matches[comment_id] = {
                "ts_variantti_matches": matches_ts,
                "ht_variantti_matches": matches_ht,
                "tt_variantti_matches": matches_tt
            }
    return pattern_matches

with open("kommentit.json", "r") as json_file:
    comments_data = json.load(json_file)

result = search_patterns_in_comments(comments_data)

for comment_id, matches in result.items():
    print(f"Comment ID: {comment_id}")
    print(f"ts_variantti matches: {matches['ts_variantti_matches']}")
    print(f"ht_variantti matches: {matches['ht_variantti_matches']}")
    print(f"tt_variantti matches: {matches['tt_variantti_matches']}\n")